In [1]:
# List of packages
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

import os, uuid
import azureml.core
#from tqdm import tqdm_notebook, tnrange
#from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
from azureml.core import Workspace, Dataset, Datastore

import random

import albumentations as A

import segmentation_models as sm
sm.set_framework('tf.keras')
from P8_01_GenerateurData_class import GenerateurData
from P8_01_GenerateurDatasasansaug_class import GenerateurDatasansaug

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [2]:
print(tf.keras.__version__)

2.4.0


In [3]:
import azureml.core
print(azureml.core.VERSION)

1.28.0


In [4]:
print(tf.version.VERSION)

2.4.0


# Données images

In [5]:
train = os.listdir('./train/')

In [6]:
train

['aachen_000000_000019_leftImg8bit.png',
 'aachen_000001_000019_leftImg8bit.png',
 'aachen_000002_000019_leftImg8bit.png',
 'aachen_000003_000019_leftImg8bit.png',
 'aachen_000004_000019_leftImg8bit.png',
 'aachen_000005_000019_leftImg8bit.png',
 'aachen_000006_000019_leftImg8bit.png',
 'aachen_000007_000019_leftImg8bit.png',
 'aachen_000008_000019_leftImg8bit.png',
 'aachen_000009_000019_leftImg8bit.png',
 'aachen_000010_000019_leftImg8bit.png',
 'aachen_000011_000019_leftImg8bit.png',
 'aachen_000012_000019_leftImg8bit.png',
 'aachen_000013_000019_leftImg8bit.png',
 'aachen_000014_000019_leftImg8bit.png',
 'aachen_000015_000019_leftImg8bit.png',
 'aachen_000016_000019_leftImg8bit.png',
 'aachen_000017_000019_leftImg8bit.png',
 'aachen_000018_000019_leftImg8bit.png',
 'aachen_000019_000019_leftImg8bit.png',
 'aachen_000020_000019_leftImg8bit.png',
 'aachen_000021_000019_leftImg8bit.png',
 'aachen_000022_000019_leftImg8bit.png',
 'aachen_000023_000019_leftImg8bit.png',
 'aachen_000024_

In [7]:
len(train)

2975

In [8]:
trainLabel = os.listdir('./trainLabel/')

In [9]:
trainLabel

['aachen_000000_000019_gtFine_labelIds.png',
 'aachen_000001_000019_gtFine_labelIds.png',
 'aachen_000002_000019_gtFine_labelIds.png',
 'aachen_000003_000019_gtFine_labelIds.png',
 'aachen_000004_000019_gtFine_labelIds.png',
 'aachen_000005_000019_gtFine_labelIds.png',
 'aachen_000006_000019_gtFine_labelIds.png',
 'aachen_000007_000019_gtFine_labelIds.png',
 'aachen_000008_000019_gtFine_labelIds.png',
 'aachen_000009_000019_gtFine_labelIds.png',
 'aachen_000010_000019_gtFine_labelIds.png',
 'aachen_000011_000019_gtFine_labelIds.png',
 'aachen_000012_000019_gtFine_labelIds.png',
 'aachen_000013_000019_gtFine_labelIds.png',
 'aachen_000014_000019_gtFine_labelIds.png',
 'aachen_000015_000019_gtFine_labelIds.png',
 'aachen_000016_000019_gtFine_labelIds.png',
 'aachen_000017_000019_gtFine_labelIds.png',
 'aachen_000018_000019_gtFine_labelIds.png',
 'aachen_000019_000019_gtFine_labelIds.png',
 'aachen_000020_000019_gtFine_labelIds.png',
 'aachen_000021_000019_gtFine_labelIds.png',
 'aachen_0

In [10]:
len(trainLabel)

2975

In [11]:
val = os.listdir('./val/')

In [12]:
val

['frankfurt_000000_000294_leftImg8bit.png',
 'frankfurt_000000_000576_leftImg8bit.png',
 'frankfurt_000000_001016_leftImg8bit.png',
 'frankfurt_000000_001236_leftImg8bit.png',
 'frankfurt_000000_001751_leftImg8bit.png',
 'frankfurt_000000_002196_leftImg8bit.png',
 'frankfurt_000000_002963_leftImg8bit.png',
 'frankfurt_000000_003025_leftImg8bit.png',
 'frankfurt_000000_003357_leftImg8bit.png',
 'frankfurt_000000_003920_leftImg8bit.png',
 'frankfurt_000000_004617_leftImg8bit.png',
 'frankfurt_000000_005543_leftImg8bit.png',
 'frankfurt_000000_005898_leftImg8bit.png',
 'frankfurt_000000_006589_leftImg8bit.png',
 'frankfurt_000000_007365_leftImg8bit.png',
 'frankfurt_000000_008206_leftImg8bit.png',
 'frankfurt_000000_008451_leftImg8bit.png',
 'frankfurt_000000_009291_leftImg8bit.png',
 'frankfurt_000000_009561_leftImg8bit.png',
 'frankfurt_000000_009688_leftImg8bit.png',
 'frankfurt_000000_009969_leftImg8bit.png',
 'frankfurt_000000_010351_leftImg8bit.png',
 'frankfurt_000000_010763_leftIm

In [13]:
len(val)

500

In [14]:
valLabel = os.listdir('./valLabel/')

In [15]:
valLabel

['frankfurt_000000_000294_gtFine_labelIds.png',
 'frankfurt_000000_000576_gtFine_labelIds.png',
 'frankfurt_000000_001016_gtFine_labelIds.png',
 'frankfurt_000000_001236_gtFine_labelIds.png',
 'frankfurt_000000_001751_gtFine_labelIds.png',
 'frankfurt_000000_002196_gtFine_labelIds.png',
 'frankfurt_000000_002963_gtFine_labelIds.png',
 'frankfurt_000000_003025_gtFine_labelIds.png',
 'frankfurt_000000_003357_gtFine_labelIds.png',
 'frankfurt_000000_003920_gtFine_labelIds.png',
 'frankfurt_000000_004617_gtFine_labelIds.png',
 'frankfurt_000000_005543_gtFine_labelIds.png',
 'frankfurt_000000_005898_gtFine_labelIds.png',
 'frankfurt_000000_006589_gtFine_labelIds.png',
 'frankfurt_000000_007365_gtFine_labelIds.png',
 'frankfurt_000000_008206_gtFine_labelIds.png',
 'frankfurt_000000_008451_gtFine_labelIds.png',
 'frankfurt_000000_009291_gtFine_labelIds.png',
 'frankfurt_000000_009561_gtFine_labelIds.png',
 'frankfurt_000000_009688_gtFine_labelIds.png',
 'frankfurt_000000_009969_gtFine_labelId

In [16]:
len(valLabel)

500

In [17]:
trainColor = os.listdir('./trainColor/')

In [18]:
trainColor

['aachen_000000_000019_gtFine_color.png',
 'aachen_000001_000019_gtFine_color.png',
 'aachen_000002_000019_gtFine_color.png',
 'aachen_000003_000019_gtFine_color.png',
 'aachen_000004_000019_gtFine_color.png',
 'aachen_000005_000019_gtFine_color.png',
 'aachen_000006_000019_gtFine_color.png',
 'aachen_000007_000019_gtFine_color.png',
 'aachen_000008_000019_gtFine_color.png',
 'aachen_000009_000019_gtFine_color.png',
 'aachen_000010_000019_gtFine_color.png',
 'aachen_000011_000019_gtFine_color.png',
 'aachen_000012_000019_gtFine_color.png',
 'aachen_000013_000019_gtFine_color.png',
 'aachen_000014_000019_gtFine_color.png',
 'aachen_000015_000019_gtFine_color.png',
 'aachen_000016_000019_gtFine_color.png',
 'aachen_000017_000019_gtFine_color.png',
 'aachen_000018_000019_gtFine_color.png',
 'aachen_000019_000019_gtFine_color.png',
 'aachen_000020_000019_gtFine_color.png',
 'aachen_000021_000019_gtFine_color.png',
 'aachen_000022_000019_gtFine_color.png',
 'aachen_000023_000019_gtFine_colo

In [19]:
len(trainColor)

2975

In [20]:
valColor = os.listdir('./valColor/')

In [21]:
valColor

['frankfurt_000000_000294_gtFine_color.png',
 'frankfurt_000000_000576_gtFine_color.png',
 'frankfurt_000000_001016_gtFine_color.png',
 'frankfurt_000000_001236_gtFine_color.png',
 'frankfurt_000000_001751_gtFine_color.png',
 'frankfurt_000000_002196_gtFine_color.png',
 'frankfurt_000000_002963_gtFine_color.png',
 'frankfurt_000000_003025_gtFine_color.png',
 'frankfurt_000000_003357_gtFine_color.png',
 'frankfurt_000000_003920_gtFine_color.png',
 'frankfurt_000000_004617_gtFine_color.png',
 'frankfurt_000000_005543_gtFine_color.png',
 'frankfurt_000000_005898_gtFine_color.png',
 'frankfurt_000000_006589_gtFine_color.png',
 'frankfurt_000000_007365_gtFine_color.png',
 'frankfurt_000000_008206_gtFine_color.png',
 'frankfurt_000000_008451_gtFine_color.png',
 'frankfurt_000000_009291_gtFine_color.png',
 'frankfurt_000000_009561_gtFine_color.png',
 'frankfurt_000000_009688_gtFine_color.png',
 'frankfurt_000000_009969_gtFine_color.png',
 'frankfurt_000000_010351_gtFine_color.png',
 'frankfur

In [22]:
len(valColor)

500

## Resize images

In [23]:
# function for resizing and cropping to 256x256
def resizeAndCrop(imgPath):

    im = Image.open(imgPath)

    # remove original
    #os.remove(imgPath)

    # Get size
    x, y = im.size

    # New sizes
    yNew = 256
    xNew = yNew # should be equal

    # First, set right size
    if x > y:
        # Y is smallest, figure out relation to 256
        xNew = round(x * 256 / y)
    else:
        yNew = round(y * 256 / x)

    # resize
    im2 = im.resize((int(xNew), int(yNew)), Image.ANTIALIAS)

    # crop
    im2.crop(((int(xNew) - 256)/2, (int(yNew) - 256)/2, (int(xNew) + 256)/2, (int(yNew) + 256)/2))

    # save
    print("SAVE", imgPath)
    im2.save(imgPath)

In [24]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
for i in range(len(train)):
    resizeAndCrop('./train/'+ train[i])

SAVE ./train/aachen_000000_000019_leftImg8bit.png
SAVE ./train/aachen_000001_000019_leftImg8bit.png
SAVE ./train/aachen_000002_000019_leftImg8bit.png
SAVE ./train/aachen_000003_000019_leftImg8bit.png
SAVE ./train/aachen_000004_000019_leftImg8bit.png
SAVE ./train/aachen_000005_000019_leftImg8bit.png
SAVE ./train/aachen_000006_000019_leftImg8bit.png
SAVE ./train/aachen_000007_000019_leftImg8bit.png
SAVE ./train/aachen_000008_000019_leftImg8bit.png
SAVE ./train/aachen_000009_000019_leftImg8bit.png
SAVE ./train/aachen_000010_000019_leftImg8bit.png
SAVE ./train/aachen_000011_000019_leftImg8bit.png
SAVE ./train/aachen_000012_000019_leftImg8bit.png
SAVE ./train/aachen_000013_000019_leftImg8bit.png
SAVE ./train/aachen_000014_000019_leftImg8bit.png
SAVE ./train/aachen_000015_000019_leftImg8bit.png
SAVE ./train/aachen_000016_000019_leftImg8bit.png
SAVE ./train/aachen_000017_000019_leftImg8bit.png
SAVE ./train/aachen_000018_000019_leftImg8bit.png
SAVE ./train/aachen_000019_000019_leftImg8bit.png


SAVE ./train/aachen_000164_000019_leftImg8bit.png
SAVE ./train/aachen_000165_000019_leftImg8bit.png
SAVE ./train/aachen_000166_000019_leftImg8bit.png
SAVE ./train/aachen_000167_000019_leftImg8bit.png
SAVE ./train/aachen_000168_000019_leftImg8bit.png
SAVE ./train/aachen_000169_000019_leftImg8bit.png
SAVE ./train/aachen_000170_000019_leftImg8bit.png
SAVE ./train/aachen_000171_000019_leftImg8bit.png
SAVE ./train/aachen_000172_000019_leftImg8bit.png
SAVE ./train/aachen_000173_000019_leftImg8bit.png
SAVE ./train/bochum_000000_000313_leftImg8bit.png
SAVE ./train/bochum_000000_000600_leftImg8bit.png
SAVE ./train/bochum_000000_000885_leftImg8bit.png
SAVE ./train/bochum_000000_001097_leftImg8bit.png
SAVE ./train/bochum_000000_001519_leftImg8bit.png
SAVE ./train/bochum_000000_001828_leftImg8bit.png
SAVE ./train/bochum_000000_002293_leftImg8bit.png
SAVE ./train/bochum_000000_002562_leftImg8bit.png
SAVE ./train/bochum_000000_003005_leftImg8bit.png
SAVE ./train/bochum_000000_003245_leftImg8bit.png


SAVE ./train/bremen_000058_000019_leftImg8bit.png
SAVE ./train/bremen_000059_000019_leftImg8bit.png
SAVE ./train/bremen_000060_000019_leftImg8bit.png
SAVE ./train/bremen_000061_000019_leftImg8bit.png
SAVE ./train/bremen_000062_000019_leftImg8bit.png
SAVE ./train/bremen_000063_000019_leftImg8bit.png
SAVE ./train/bremen_000064_000019_leftImg8bit.png
SAVE ./train/bremen_000065_000019_leftImg8bit.png
SAVE ./train/bremen_000066_000019_leftImg8bit.png
SAVE ./train/bremen_000067_000019_leftImg8bit.png
SAVE ./train/bremen_000068_000019_leftImg8bit.png
SAVE ./train/bremen_000069_000019_leftImg8bit.png
SAVE ./train/bremen_000070_000019_leftImg8bit.png
SAVE ./train/bremen_000071_000019_leftImg8bit.png
SAVE ./train/bremen_000072_000019_leftImg8bit.png
SAVE ./train/bremen_000073_000019_leftImg8bit.png
SAVE ./train/bremen_000074_000019_leftImg8bit.png
SAVE ./train/bremen_000075_000019_leftImg8bit.png
SAVE ./train/bremen_000076_000019_leftImg8bit.png
SAVE ./train/bremen_000077_000019_leftImg8bit.png


SAVE ./train/bremen_000223_000019_leftImg8bit.png
SAVE ./train/bremen_000224_000019_leftImg8bit.png
SAVE ./train/bremen_000225_000019_leftImg8bit.png
SAVE ./train/bremen_000226_000019_leftImg8bit.png
SAVE ./train/bremen_000227_000019_leftImg8bit.png
SAVE ./train/bremen_000228_000019_leftImg8bit.png
SAVE ./train/bremen_000229_000019_leftImg8bit.png
SAVE ./train/bremen_000230_000019_leftImg8bit.png
SAVE ./train/bremen_000231_000019_leftImg8bit.png
SAVE ./train/bremen_000232_000019_leftImg8bit.png
SAVE ./train/bremen_000233_000019_leftImg8bit.png
SAVE ./train/bremen_000234_000019_leftImg8bit.png
SAVE ./train/bremen_000235_000019_leftImg8bit.png
SAVE ./train/bremen_000236_000019_leftImg8bit.png
SAVE ./train/bremen_000237_000019_leftImg8bit.png
SAVE ./train/bremen_000238_000019_leftImg8bit.png
SAVE ./train/bremen_000239_000019_leftImg8bit.png
SAVE ./train/bremen_000240_000019_leftImg8bit.png
SAVE ./train/bremen_000241_000019_leftImg8bit.png
SAVE ./train/bremen_000242_000019_leftImg8bit.png


SAVE ./train/cologne_000072_000019_leftImg8bit.png
SAVE ./train/cologne_000073_000019_leftImg8bit.png
SAVE ./train/cologne_000074_000019_leftImg8bit.png
SAVE ./train/cologne_000075_000019_leftImg8bit.png
SAVE ./train/cologne_000076_000019_leftImg8bit.png
SAVE ./train/cologne_000077_000019_leftImg8bit.png
SAVE ./train/cologne_000078_000019_leftImg8bit.png
SAVE ./train/cologne_000079_000019_leftImg8bit.png
SAVE ./train/cologne_000080_000019_leftImg8bit.png
SAVE ./train/cologne_000081_000019_leftImg8bit.png
SAVE ./train/cologne_000082_000019_leftImg8bit.png
SAVE ./train/cologne_000083_000019_leftImg8bit.png
SAVE ./train/cologne_000084_000019_leftImg8bit.png
SAVE ./train/cologne_000085_000019_leftImg8bit.png
SAVE ./train/cologne_000086_000019_leftImg8bit.png
SAVE ./train/cologne_000087_000019_leftImg8bit.png
SAVE ./train/cologne_000088_000019_leftImg8bit.png
SAVE ./train/cologne_000089_000019_leftImg8bit.png
SAVE ./train/cologne_000090_000019_leftImg8bit.png
SAVE ./train/cologne_000091_000

...

SAVE ./train/strasbourg_000001_052979_leftImg8bit.png
SAVE ./train/strasbourg_000001_053222_leftImg8bit.png
SAVE ./train/strasbourg_000001_053579_leftImg8bit.png
SAVE ./train/strasbourg_000001_053976_leftImg8bit.png
SAVE ./train/strasbourg_000001_054275_leftImg8bit.png
SAVE ./train/strasbourg_000001_054639_leftImg8bit.png
SAVE ./train/strasbourg_000001_055273_leftImg8bit.png
SAVE ./train/strasbourg_000001_055698_leftImg8bit.png
SAVE ./train/strasbourg_000001_055860_leftImg8bit.png
SAVE ./train/strasbourg_000001_055934_leftImg8bit.png
SAVE ./train/strasbourg_000001_056142_leftImg8bit.png
SAVE ./train/strasbourg_000001_056330_leftImg8bit.png
SAVE ./train/strasbourg_000001_056857_leftImg8bit.png
SAVE ./train/strasbourg_000001_057129_leftImg8bit.png
SAVE ./train/strasbourg_000001_057191_leftImg8bit.png
SAVE ./train/strasbourg_000001_057517_leftImg8bit.png
SAVE ./train/strasbourg_000001_057811_leftImg8bit.png
SAVE ./train/strasbourg_000001_057930_leftImg8bit.png
SAVE ./train/strasbourg_0000

SAVE ./train/stuttgart_000115_000019_leftImg8bit.png
SAVE ./train/stuttgart_000116_000019_leftImg8bit.png
SAVE ./train/stuttgart_000117_000019_leftImg8bit.png
SAVE ./train/stuttgart_000118_000019_leftImg8bit.png
SAVE ./train/stuttgart_000119_000019_leftImg8bit.png
SAVE ./train/stuttgart_000120_000019_leftImg8bit.png
SAVE ./train/stuttgart_000121_000019_leftImg8bit.png
SAVE ./train/stuttgart_000122_000019_leftImg8bit.png
SAVE ./train/stuttgart_000123_000019_leftImg8bit.png
SAVE ./train/stuttgart_000124_000019_leftImg8bit.png
SAVE ./train/stuttgart_000125_000019_leftImg8bit.png
SAVE ./train/stuttgart_000126_000019_leftImg8bit.png
SAVE ./train/stuttgart_000127_000019_leftImg8bit.png
SAVE ./train/stuttgart_000128_000019_leftImg8bit.png
SAVE ./train/stuttgart_000129_000019_leftImg8bit.png
SAVE ./train/stuttgart_000130_000019_leftImg8bit.png
SAVE ./train/stuttgart_000131_000019_leftImg8bit.png
SAVE ./train/stuttgart_000132_000019_leftImg8bit.png
SAVE ./train/stuttgart_000133_000019_leftImg8b

SAVE ./train/tubingen_000076_000019_leftImg8bit.png
SAVE ./train/tubingen_000077_000019_leftImg8bit.png
SAVE ./train/tubingen_000078_000019_leftImg8bit.png
SAVE ./train/tubingen_000079_000019_leftImg8bit.png
SAVE ./train/tubingen_000080_000019_leftImg8bit.png
SAVE ./train/tubingen_000081_000019_leftImg8bit.png
SAVE ./train/tubingen_000082_000019_leftImg8bit.png
SAVE ./train/tubingen_000083_000019_leftImg8bit.png
SAVE ./train/tubingen_000084_000019_leftImg8bit.png
SAVE ./train/tubingen_000085_000019_leftImg8bit.png
SAVE ./train/tubingen_000086_000019_leftImg8bit.png
SAVE ./train/tubingen_000087_000019_leftImg8bit.png
SAVE ./train/tubingen_000088_000019_leftImg8bit.png
SAVE ./train/tubingen_000089_000019_leftImg8bit.png
SAVE ./train/tubingen_000090_000019_leftImg8bit.png
SAVE ./train/tubingen_000091_000019_leftImg8bit.png
SAVE ./train/tubingen_000092_000019_leftImg8bit.png
SAVE ./train/tubingen_000093_000019_leftImg8bit.png
SAVE ./train/tubingen_000094_000019_leftImg8bit.png
SAVE ./train

SAVE ./train/weimar_000006_000019_leftImg8bit.png
SAVE ./train/weimar_000007_000019_leftImg8bit.png
SAVE ./train/weimar_000008_000019_leftImg8bit.png
SAVE ./train/weimar_000009_000019_leftImg8bit.png
SAVE ./train/weimar_000010_000019_leftImg8bit.png
SAVE ./train/weimar_000011_000019_leftImg8bit.png
SAVE ./train/weimar_000012_000019_leftImg8bit.png
SAVE ./train/weimar_000013_000019_leftImg8bit.png
SAVE ./train/weimar_000014_000019_leftImg8bit.png
SAVE ./train/weimar_000015_000019_leftImg8bit.png
SAVE ./train/weimar_000016_000019_leftImg8bit.png
SAVE ./train/weimar_000017_000019_leftImg8bit.png
SAVE ./train/weimar_000018_000019_leftImg8bit.png
SAVE ./train/weimar_000019_000019_leftImg8bit.png
SAVE ./train/weimar_000020_000019_leftImg8bit.png
SAVE ./train/weimar_000021_000019_leftImg8bit.png
SAVE ./train/weimar_000022_000019_leftImg8bit.png
SAVE ./train/weimar_000023_000019_leftImg8bit.png
SAVE ./train/weimar_000024_000019_leftImg8bit.png
SAVE ./train/weimar_000025_000019_leftImg8bit.png


SAVE ./train/zurich_000029_000019_leftImg8bit.png
SAVE ./train/zurich_000030_000019_leftImg8bit.png
SAVE ./train/zurich_000031_000019_leftImg8bit.png
SAVE ./train/zurich_000032_000019_leftImg8bit.png
SAVE ./train/zurich_000033_000019_leftImg8bit.png
SAVE ./train/zurich_000034_000019_leftImg8bit.png
SAVE ./train/zurich_000035_000019_leftImg8bit.png
SAVE ./train/zurich_000036_000019_leftImg8bit.png
SAVE ./train/zurich_000037_000019_leftImg8bit.png
SAVE ./train/zurich_000038_000019_leftImg8bit.png
SAVE ./train/zurich_000039_000019_leftImg8bit.png
SAVE ./train/zurich_000040_000019_leftImg8bit.png
SAVE ./train/zurich_000041_000019_leftImg8bit.png
SAVE ./train/zurich_000042_000019_leftImg8bit.png
SAVE ./train/zurich_000043_000019_leftImg8bit.png
SAVE ./train/zurich_000044_000019_leftImg8bit.png
SAVE ./train/zurich_000045_000019_leftImg8bit.png
SAVE ./train/zurich_000046_000019_leftImg8bit.png
SAVE ./train/zurich_000047_000019_leftImg8bit.png
SAVE ./train/zurich_000048_000019_leftImg8bit.png


In [25]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
for i in range(len(val)):
    resizeAndCrop('./val/'+ val[i])

SAVE ./val/frankfurt_000000_000294_leftImg8bit.png
SAVE ./val/frankfurt_000000_000576_leftImg8bit.png
SAVE ./val/frankfurt_000000_001016_leftImg8bit.png
SAVE ./val/frankfurt_000000_001236_leftImg8bit.png
SAVE ./val/frankfurt_000000_001751_leftImg8bit.png
SAVE ./val/frankfurt_000000_002196_leftImg8bit.png
SAVE ./val/frankfurt_000000_002963_leftImg8bit.png
SAVE ./val/frankfurt_000000_003025_leftImg8bit.png
SAVE ./val/frankfurt_000000_003357_leftImg8bit.png
SAVE ./val/frankfurt_000000_003920_leftImg8bit.png
SAVE ./val/frankfurt_000000_004617_leftImg8bit.png
SAVE ./val/frankfurt_000000_005543_leftImg8bit.png
SAVE ./val/frankfurt_000000_005898_leftImg8bit.png
SAVE ./val/frankfurt_000000_006589_leftImg8bit.png
SAVE ./val/frankfurt_000000_007365_leftImg8bit.png
SAVE ./val/frankfurt_000000_008206_leftImg8bit.png
SAVE ./val/frankfurt_000000_008451_leftImg8bit.png
SAVE ./val/frankfurt_000000_009291_leftImg8bit.png
SAVE ./val/frankfurt_000000_009561_leftImg8bit.png
SAVE ./val/frankfurt_000000_009

SAVE ./val/frankfurt_000001_046126_leftImg8bit.png
SAVE ./val/frankfurt_000001_046272_leftImg8bit.png
SAVE ./val/frankfurt_000001_046504_leftImg8bit.png
SAVE ./val/frankfurt_000001_046779_leftImg8bit.png
SAVE ./val/frankfurt_000001_047178_leftImg8bit.png
SAVE ./val/frankfurt_000001_047552_leftImg8bit.png
SAVE ./val/frankfurt_000001_048196_leftImg8bit.png
SAVE ./val/frankfurt_000001_048355_leftImg8bit.png
SAVE ./val/frankfurt_000001_048654_leftImg8bit.png
SAVE ./val/frankfurt_000001_049078_leftImg8bit.png
SAVE ./val/frankfurt_000001_049209_leftImg8bit.png
SAVE ./val/frankfurt_000001_049298_leftImg8bit.png
SAVE ./val/frankfurt_000001_049698_leftImg8bit.png
SAVE ./val/frankfurt_000001_049770_leftImg8bit.png
SAVE ./val/frankfurt_000001_050149_leftImg8bit.png
SAVE ./val/frankfurt_000001_050686_leftImg8bit.png
SAVE ./val/frankfurt_000001_051516_leftImg8bit.png
SAVE ./val/frankfurt_000001_051737_leftImg8bit.png
SAVE ./val/frankfurt_000001_051807_leftImg8bit.png
SAVE ./val/frankfurt_000001_052

SAVE ./val/munster_000001_000019_leftImg8bit.png
SAVE ./val/munster_000002_000019_leftImg8bit.png
SAVE ./val/munster_000003_000019_leftImg8bit.png
SAVE ./val/munster_000004_000019_leftImg8bit.png
SAVE ./val/munster_000005_000019_leftImg8bit.png
SAVE ./val/munster_000006_000019_leftImg8bit.png
SAVE ./val/munster_000007_000019_leftImg8bit.png
SAVE ./val/munster_000008_000019_leftImg8bit.png
SAVE ./val/munster_000009_000019_leftImg8bit.png
SAVE ./val/munster_000010_000019_leftImg8bit.png
SAVE ./val/munster_000011_000019_leftImg8bit.png
SAVE ./val/munster_000012_000019_leftImg8bit.png
SAVE ./val/munster_000013_000019_leftImg8bit.png
SAVE ./val/munster_000014_000019_leftImg8bit.png
SAVE ./val/munster_000015_000019_leftImg8bit.png
SAVE ./val/munster_000016_000019_leftImg8bit.png
SAVE ./val/munster_000017_000019_leftImg8bit.png
SAVE ./val/munster_000018_000019_leftImg8bit.png
SAVE ./val/munster_000019_000019_leftImg8bit.png
SAVE ./val/munster_000020_000019_leftImg8bit.png
SAVE ./val/munster_0

SAVE ./val/munster_000170_000019_leftImg8bit.png
SAVE ./val/munster_000171_000019_leftImg8bit.png
SAVE ./val/munster_000172_000019_leftImg8bit.png
SAVE ./val/munster_000173_000019_leftImg8bit.png


In [26]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
for i in range(len(trainColor)):
    resizeAndCrop('./trainColor/'+ trainColor[i])

SAVE ./trainColor/aachen_000000_000019_gtFine_color.png
SAVE ./trainColor/aachen_000001_000019_gtFine_color.png
SAVE ./trainColor/aachen_000002_000019_gtFine_color.png
SAVE ./trainColor/aachen_000003_000019_gtFine_color.png
SAVE ./trainColor/aachen_000004_000019_gtFine_color.png
SAVE ./trainColor/aachen_000005_000019_gtFine_color.png
SAVE ./trainColor/aachen_000006_000019_gtFine_color.png
SAVE ./trainColor/aachen_000007_000019_gtFine_color.png
SAVE ./trainColor/aachen_000008_000019_gtFine_color.png
SAVE ./trainColor/aachen_000009_000019_gtFine_color.png
SAVE ./trainColor/aachen_000010_000019_gtFine_color.png
SAVE ./trainColor/aachen_000011_000019_gtFine_color.png
SAVE ./trainColor/aachen_000012_000019_gtFine_color.png
SAVE ./trainColor/aachen_000013_000019_gtFine_color.png
SAVE ./trainColor/aachen_000014_000019_gtFine_color.png
SAVE ./trainColor/aachen_000015_000019_gtFine_color.png
SAVE ./trainColor/aachen_000016_000019_gtFine_color.png
SAVE ./trainColor/aachen_000017_000019_gtFine_co

SAVE ./trainColor/aachen_000147_000019_gtFine_color.png
SAVE ./trainColor/aachen_000148_000019_gtFine_color.png
SAVE ./trainColor/aachen_000149_000019_gtFine_color.png
SAVE ./trainColor/aachen_000150_000019_gtFine_color.png
SAVE ./trainColor/aachen_000151_000019_gtFine_color.png
SAVE ./trainColor/aachen_000152_000019_gtFine_color.png
SAVE ./trainColor/aachen_000153_000019_gtFine_color.png
SAVE ./trainColor/aachen_000154_000019_gtFine_color.png
SAVE ./trainColor/aachen_000155_000019_gtFine_color.png
SAVE ./trainColor/aachen_000156_000019_gtFine_color.png
SAVE ./trainColor/aachen_000157_000019_gtFine_color.png
SAVE ./trainColor/aachen_000158_000019_gtFine_color.png
SAVE ./trainColor/aachen_000159_000019_gtFine_color.png
SAVE ./trainColor/aachen_000160_000019_gtFine_color.png
SAVE ./trainColor/aachen_000161_000019_gtFine_color.png
SAVE ./trainColor/aachen_000162_000019_gtFine_color.png
SAVE ./trainColor/aachen_000163_000019_gtFine_color.png
SAVE ./trainColor/aachen_000164_000019_gtFine_co

SAVE ./trainColor/bremen_000026_000019_gtFine_color.png
SAVE ./trainColor/bremen_000027_000019_gtFine_color.png
SAVE ./trainColor/bremen_000028_000019_gtFine_color.png
SAVE ./trainColor/bremen_000029_000019_gtFine_color.png
SAVE ./trainColor/bremen_000030_000019_gtFine_color.png
SAVE ./trainColor/bremen_000031_000019_gtFine_color.png
SAVE ./trainColor/bremen_000032_000019_gtFine_color.png
SAVE ./trainColor/bremen_000033_000019_gtFine_color.png
SAVE ./trainColor/bremen_000034_000019_gtFine_color.png
SAVE ./trainColor/bremen_000035_000019_gtFine_color.png
SAVE ./trainColor/bremen_000036_000019_gtFine_color.png
SAVE ./trainColor/bremen_000037_000019_gtFine_color.png
SAVE ./trainColor/bremen_000038_000019_gtFine_color.png
SAVE ./trainColor/bremen_000039_000019_gtFine_color.png
SAVE ./trainColor/bremen_000040_000019_gtFine_color.png
SAVE ./trainColor/bremen_000041_000019_gtFine_color.png
SAVE ./trainColor/bremen_000042_000019_gtFine_color.png
SAVE ./trainColor/bremen_000043_000019_gtFine_co

SAVE ./trainColor/bremen_000174_000019_gtFine_color.png
SAVE ./trainColor/bremen_000175_000019_gtFine_color.png
SAVE ./trainColor/bremen_000176_000019_gtFine_color.png
SAVE ./trainColor/bremen_000177_000019_gtFine_color.png
SAVE ./trainColor/bremen_000178_000019_gtFine_color.png
SAVE ./trainColor/bremen_000179_000019_gtFine_color.png
SAVE ./trainColor/bremen_000180_000019_gtFine_color.png
SAVE ./trainColor/bremen_000181_000019_gtFine_color.png
SAVE ./trainColor/bremen_000182_000019_gtFine_color.png
SAVE ./trainColor/bremen_000183_000019_gtFine_color.png
SAVE ./trainColor/bremen_000184_000019_gtFine_color.png
SAVE ./trainColor/bremen_000185_000019_gtFine_color.png
SAVE ./trainColor/bremen_000186_000019_gtFine_color.png
SAVE ./trainColor/bremen_000187_000019_gtFine_color.png
SAVE ./trainColor/bremen_000188_000019_gtFine_color.png
SAVE ./trainColor/bremen_000189_000019_gtFine_color.png
SAVE ./trainColor/bremen_000190_000019_gtFine_color.png
SAVE ./trainColor/bremen_000191_000019_gtFine_co

SAVE ./trainColor/cologne_000005_000019_gtFine_color.png
SAVE ./trainColor/cologne_000006_000019_gtFine_color.png
SAVE ./trainColor/cologne_000007_000019_gtFine_color.png
SAVE ./trainColor/cologne_000008_000019_gtFine_color.png
SAVE ./trainColor/cologne_000009_000019_gtFine_color.png
SAVE ./trainColor/cologne_000010_000019_gtFine_color.png
SAVE ./trainColor/cologne_000011_000019_gtFine_color.png
SAVE ./trainColor/cologne_000012_000019_gtFine_color.png
SAVE ./trainColor/cologne_000013_000019_gtFine_color.png
SAVE ./trainColor/cologne_000014_000019_gtFine_color.png
SAVE ./trainColor/cologne_000015_000019_gtFine_color.png
SAVE ./trainColor/cologne_000016_000019_gtFine_color.png
SAVE ./trainColor/cologne_000017_000019_gtFine_color.png
SAVE ./trainColor/cologne_000018_000019_gtFine_color.png
SAVE ./trainColor/cologne_000019_000019_gtFine_color.png
SAVE ./trainColor/cologne_000020_000019_gtFine_color.png
SAVE ./trainColor/cologne_000021_000019_gtFine_color.png
SAVE ./trainColor/cologne_00002

...

SAVE ./trainColor/stuttgart_000013_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000014_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000015_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000016_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000017_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000018_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000019_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000020_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000021_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000022_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000023_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000024_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000025_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000026_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000027_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000028_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000029_000019_gtFine_color.p

SAVE ./trainColor/stuttgart_000153_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000154_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000155_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000156_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000157_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000158_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000159_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000160_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000161_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000162_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000163_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000164_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000165_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000166_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000167_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000168_000019_gtFine_color.png
SAVE ./trainColor/stuttgart_000169_000019_gtFine_color.p

SAVE ./trainColor/tubingen_000098_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000099_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000100_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000101_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000102_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000103_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000104_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000105_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000106_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000107_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000108_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000109_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000110_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000111_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000112_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000113_000019_gtFine_color.png
SAVE ./trainColor/tubingen_000114_000019_gtFine_color.png
SAVE ./trainCo

SAVE ./trainColor/weimar_000010_000019_gtFine_color.png
SAVE ./trainColor/weimar_000011_000019_gtFine_color.png
SAVE ./trainColor/weimar_000012_000019_gtFine_color.png
SAVE ./trainColor/weimar_000013_000019_gtFine_color.png
SAVE ./trainColor/weimar_000014_000019_gtFine_color.png
SAVE ./trainColor/weimar_000015_000019_gtFine_color.png
SAVE ./trainColor/weimar_000016_000019_gtFine_color.png
SAVE ./trainColor/weimar_000017_000019_gtFine_color.png
SAVE ./trainColor/weimar_000018_000019_gtFine_color.png
SAVE ./trainColor/weimar_000019_000019_gtFine_color.png
SAVE ./trainColor/weimar_000020_000019_gtFine_color.png
SAVE ./trainColor/weimar_000021_000019_gtFine_color.png
SAVE ./trainColor/weimar_000022_000019_gtFine_color.png
SAVE ./trainColor/weimar_000023_000019_gtFine_color.png
SAVE ./trainColor/weimar_000024_000019_gtFine_color.png
SAVE ./trainColor/weimar_000025_000019_gtFine_color.png
SAVE ./trainColor/weimar_000026_000019_gtFine_color.png
SAVE ./trainColor/weimar_000027_000019_gtFine_co

SAVE ./trainColor/zurich_000016_000019_gtFine_color.png
SAVE ./trainColor/zurich_000017_000019_gtFine_color.png
SAVE ./trainColor/zurich_000018_000019_gtFine_color.png
SAVE ./trainColor/zurich_000019_000019_gtFine_color.png
SAVE ./trainColor/zurich_000020_000019_gtFine_color.png
SAVE ./trainColor/zurich_000021_000019_gtFine_color.png
SAVE ./trainColor/zurich_000022_000019_gtFine_color.png
SAVE ./trainColor/zurich_000023_000019_gtFine_color.png
SAVE ./trainColor/zurich_000024_000019_gtFine_color.png
SAVE ./trainColor/zurich_000025_000019_gtFine_color.png
SAVE ./trainColor/zurich_000026_000019_gtFine_color.png
SAVE ./trainColor/zurich_000027_000019_gtFine_color.png
SAVE ./trainColor/zurich_000028_000019_gtFine_color.png
SAVE ./trainColor/zurich_000029_000019_gtFine_color.png
SAVE ./trainColor/zurich_000030_000019_gtFine_color.png
SAVE ./trainColor/zurich_000031_000019_gtFine_color.png
SAVE ./trainColor/zurich_000032_000019_gtFine_color.png
SAVE ./trainColor/zurich_000033_000019_gtFine_co

In [27]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
for i in range(len(valColor)):
    resizeAndCrop('./valColor/'+ valColor[i])

SAVE ./valColor/frankfurt_000000_000294_gtFine_color.png
SAVE ./valColor/frankfurt_000000_000576_gtFine_color.png
SAVE ./valColor/frankfurt_000000_001016_gtFine_color.png
SAVE ./valColor/frankfurt_000000_001236_gtFine_color.png
SAVE ./valColor/frankfurt_000000_001751_gtFine_color.png
SAVE ./valColor/frankfurt_000000_002196_gtFine_color.png
SAVE ./valColor/frankfurt_000000_002963_gtFine_color.png
SAVE ./valColor/frankfurt_000000_003025_gtFine_color.png
SAVE ./valColor/frankfurt_000000_003357_gtFine_color.png
SAVE ./valColor/frankfurt_000000_003920_gtFine_color.png
SAVE ./valColor/frankfurt_000000_004617_gtFine_color.png
SAVE ./valColor/frankfurt_000000_005543_gtFine_color.png
SAVE ./valColor/frankfurt_000000_005898_gtFine_color.png
SAVE ./valColor/frankfurt_000000_006589_gtFine_color.png
SAVE ./valColor/frankfurt_000000_007365_gtFine_color.png
SAVE ./valColor/frankfurt_000000_008206_gtFine_color.png
SAVE ./valColor/frankfurt_000000_008451_gtFine_color.png
SAVE ./valColor/frankfurt_00000

SAVE ./valColor/frankfurt_000001_038844_gtFine_color.png
SAVE ./valColor/frankfurt_000001_039895_gtFine_color.png
SAVE ./valColor/frankfurt_000001_040575_gtFine_color.png
SAVE ./valColor/frankfurt_000001_040732_gtFine_color.png
SAVE ./valColor/frankfurt_000001_041074_gtFine_color.png
SAVE ./valColor/frankfurt_000001_041354_gtFine_color.png
SAVE ./valColor/frankfurt_000001_041517_gtFine_color.png
SAVE ./valColor/frankfurt_000001_041664_gtFine_color.png
SAVE ./valColor/frankfurt_000001_042098_gtFine_color.png
SAVE ./valColor/frankfurt_000001_042384_gtFine_color.png
SAVE ./valColor/frankfurt_000001_042733_gtFine_color.png
SAVE ./valColor/frankfurt_000001_043395_gtFine_color.png
SAVE ./valColor/frankfurt_000001_043564_gtFine_color.png
SAVE ./valColor/frankfurt_000001_044227_gtFine_color.png
SAVE ./valColor/frankfurt_000001_044413_gtFine_color.png
SAVE ./valColor/frankfurt_000001_044525_gtFine_color.png
SAVE ./valColor/frankfurt_000001_044658_gtFine_color.png
SAVE ./valColor/frankfurt_00000

SAVE ./valColor/lindau_000022_000019_gtFine_color.png
SAVE ./valColor/lindau_000023_000019_gtFine_color.png
SAVE ./valColor/lindau_000024_000019_gtFine_color.png
SAVE ./valColor/lindau_000025_000019_gtFine_color.png
SAVE ./valColor/lindau_000026_000019_gtFine_color.png
SAVE ./valColor/lindau_000027_000019_gtFine_color.png
SAVE ./valColor/lindau_000028_000019_gtFine_color.png
SAVE ./valColor/lindau_000029_000019_gtFine_color.png
SAVE ./valColor/lindau_000030_000019_gtFine_color.png
SAVE ./valColor/lindau_000031_000019_gtFine_color.png
SAVE ./valColor/lindau_000032_000019_gtFine_color.png
SAVE ./valColor/lindau_000033_000019_gtFine_color.png
SAVE ./valColor/lindau_000034_000019_gtFine_color.png
SAVE ./valColor/lindau_000035_000019_gtFine_color.png
SAVE ./valColor/lindau_000036_000019_gtFine_color.png
SAVE ./valColor/lindau_000037_000019_gtFine_color.png
SAVE ./valColor/lindau_000038_000019_gtFine_color.png
SAVE ./valColor/lindau_000039_000019_gtFine_color.png
SAVE ./valColor/lindau_00004

SAVE ./valColor/munster_000114_000019_gtFine_color.png
SAVE ./valColor/munster_000115_000019_gtFine_color.png
SAVE ./valColor/munster_000116_000019_gtFine_color.png
SAVE ./valColor/munster_000117_000019_gtFine_color.png
SAVE ./valColor/munster_000118_000019_gtFine_color.png
SAVE ./valColor/munster_000119_000019_gtFine_color.png
SAVE ./valColor/munster_000120_000019_gtFine_color.png
SAVE ./valColor/munster_000121_000019_gtFine_color.png
SAVE ./valColor/munster_000122_000019_gtFine_color.png
SAVE ./valColor/munster_000123_000019_gtFine_color.png
SAVE ./valColor/munster_000124_000019_gtFine_color.png
SAVE ./valColor/munster_000125_000019_gtFine_color.png
SAVE ./valColor/munster_000126_000019_gtFine_color.png
SAVE ./valColor/munster_000127_000019_gtFine_color.png
SAVE ./valColor/munster_000128_000019_gtFine_color.png
SAVE ./valColor/munster_000129_000019_gtFine_color.png
SAVE ./valColor/munster_000130_000019_gtFine_color.png
SAVE ./valColor/munster_000131_000019_gtFine_color.png
SAVE ./val

In [28]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
for i in range(len(trainLabel)):
    resizeAndCrop('./trainLabel/'+ trainLabel[i])

SAVE ./trainLabel/aachen_000000_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000001_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000002_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000003_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000004_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000005_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000006_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000007_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000008_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000009_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000010_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000011_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000012_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000013_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000014_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000015_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000016_000019_gtFine_labelIds.p

SAVE ./trainLabel/aachen_000144_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000145_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000146_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000147_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000148_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000149_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000150_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000151_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000152_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000153_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000154_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000155_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000156_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000157_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000158_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000159_000019_gtFine_labelIds.png
SAVE ./trainLabel/aachen_000160_000019_gtFine_labelIds.p

SAVE ./trainLabel/bremen_000017_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000018_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000019_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000020_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000021_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000022_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000023_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000024_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000025_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000026_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000027_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000028_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000029_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000030_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000031_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000032_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000033_000019_gtFine_labelIds.p

SAVE ./trainLabel/bremen_000159_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000160_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000161_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000162_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000163_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000164_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000165_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000166_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000167_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000168_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000169_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000170_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000171_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000172_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000173_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000174_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000175_000019_gtFine_labelIds.p

SAVE ./trainLabel/bremen_000302_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000303_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000304_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000305_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000306_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000307_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000308_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000309_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000310_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000311_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000312_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000313_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000314_000019_gtFine_labelIds.png
SAVE ./trainLabel/bremen_000315_000019_gtFine_labelIds.png
SAVE ./trainLabel/cologne_000000_000019_gtFine_labelIds.png
SAVE ./trainLabel/cologne_000001_000019_gtFine_labelIds.png
SAVE ./trainLabel/cologne_000002_000019_gtFine_labelId

...

SAVE ./trainLabel/stuttgart_000090_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000091_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000092_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000093_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000094_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000095_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000096_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000097_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000098_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000099_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000100_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000101_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000102_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000103_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000104_000019_gtFine_labelIds.png
SAVE ./trainLabel/stuttgart_000105_000019_gtFine_labelIds.png
SAVE ./t

SAVE ./trainLabel/tubingen_000033_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000034_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000035_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000036_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000037_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000038_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000039_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000040_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000041_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000042_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000043_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000044_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000045_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000046_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000047_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubingen_000048_000019_gtFine_labelIds.png
SAVE ./trainLabel/tubing

SAVE ./trainLabel/ulm_000027_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000028_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000029_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000030_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000031_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000032_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000033_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000034_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000035_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000036_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000037_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000038_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000039_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000040_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000041_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000042_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000043_000019_gtFine_labelIds.png
SAVE ./trainLabel/ulm_000044_000019_gtFine_label

SAVE ./trainLabel/weimar_000075_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000076_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000077_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000078_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000079_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000080_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000081_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000082_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000083_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000084_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000085_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000086_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000087_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000088_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000089_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000090_000019_gtFine_labelIds.png
SAVE ./trainLabel/weimar_000091_000019_gtFine_labelIds.p

SAVE ./trainLabel/zurich_000073_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000074_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000075_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000076_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000077_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000078_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000079_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000080_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000081_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000082_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000083_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000084_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000085_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000086_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000087_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000088_000019_gtFine_labelIds.png
SAVE ./trainLabel/zurich_000089_000019_gtFine_labelIds.p

In [29]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
for i in range(len(valLabel)):
    resizeAndCrop('./valLabel/'+ valLabel[i])

SAVE ./valLabel/frankfurt_000000_000294_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_000576_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_001016_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_001236_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_001751_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_002196_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_002963_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_003025_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_003357_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_003920_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_004617_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_005543_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_005898_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_006589_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_007365_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_008206_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000000_008451_

SAVE ./valLabel/frankfurt_000001_035144_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_035864_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_037705_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_038245_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_038418_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_038645_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_038844_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_039895_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_040575_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_040732_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_041074_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_041354_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_041517_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_041664_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_042098_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_042384_gtFine_labelIds.png
SAVE ./valLabel/frankfurt_000001_042733_

SAVE ./valLabel/lindau_000012_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000013_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000014_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000015_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000016_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000017_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000018_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000019_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000020_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000021_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000022_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000023_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000024_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000025_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000026_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000027_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000028_000019_gtFine_labelIds.png
SAVE ./valLabel/lindau_000029_0

SAVE ./valLabel/munster_000101_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000102_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000103_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000104_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000105_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000106_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000107_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000108_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000109_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000110_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000111_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000112_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000113_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000114_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000115_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000116_000019_gtFine_labelIds.png
SAVE ./valLabel/munster_000117_000019_gtFine_labelIds.png
SAVE ./valLabe